<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/master/latent_space_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Mount google drive.
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

# Check if we have linked the folder
from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    print(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

In [ ]:
#@title Configure dataset
#@markdown - choose whether to use wandb id or use checkpoint_path to select checkpoint file
is_wandb_id = True    #@param {type:"boolean"}

#@markdown Choice 1: wandb id and project_name to select checkpoint file
#@markdown - will be ignored if is_wandb_id = False
#@markdown - set `"run_id"` to resume a run (for example: `u9imsvva`)
#@markdown - The id of the current run is shown below in the cell with `wandb.init()` or you can find it in the wandb web UI.
id = "60ukopye" #@param {type:"string"}
#@markdown - Enter project name (either `handtool-gan` or `tree-gan`)
project_name = "tree-gan" #@param ["tree-gan", "handtool-gan"]

#@markdown Choice 2: file path to select checkpoint file
#@markdown - will be ignored if is_wandb_id = True
#@markdown - For example via the file browser on the left to locate and right click to copy the path.)
#@markdown - file folder example: `/content/drive/My Drive/h/k` 
ckpt_file_location = f"/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/{id}" #@param {type:"string"}

#@markdown Enter 2 files location to specify 2 latent vectors.   
#@markdown - For example via the file browser on the left to locate and right click to copy the path.)
#@markdown - file folder example: `/content/drive/My Drive/h/k` 
file1 = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj/obj_files/old_1.obj" #@param {type:"string"}
file2 = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj/obj_files/maple_example2.obj" #@param {type:"string"}

#@markdown Enter how many samples to generate
test_num_samples = 20    #@param {type:"integer"}
#@markdown Enter export location (folder/directory).   
#@markdown - For example via the file browser on the left to locate and right click to copy the path.)
#@markdown - file folder example: `/content/drive/My Drive/h/k` 
export_location = f"/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/{id}" #@param {type:"string"}



print("is_wandb_id:", is_wandb_id)
if is_wandb_id:
    print("id:", id)
else:
    print("ckpt_file_location:", ckpt_file_location)
print("export_location:", export_location)


In [ ]:
#@markdown Install wandb and log in
if is_wandb_id:
    !pip install wandb
    output.clear()
    import wandb
    !wandb login
    run = wandb.init(project=project_name, id=id, entity="bugan", resume=True, dir="./")
    output.clear()
    print("run id: " + str(wandb.run.id))
    print("run name: " + str(wandb.run.name))
    print("ok!")

# To just train a model, no edits should be required in any cells below.

In [ ]:
import os
from pathlib import Path
os.environ["WANDB_MODE"] = "dryrun"

%cd /content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/

run_path = "./"

!apt-get update

!pip install pytorch-lightning
!pip install trimesh
!apt install -y xvfb
!pip install trimesh xvfbwrapper
#bugan package
!pip install -U git+https://github.com/buganart/BUGAN.git#egg=bugan
output.clear()
print('ok!')

In [ ]:
import io
from io import BytesIO
import zipfile
import trimesh
import numpy as np


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger


from bugan.functionsPL import *
from bugan.modelsPL import VAEGAN

# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

from xvfbwrapper import Xvfb

#load model

In [ ]:
if is_wandb_id:
    checkpoint_path = os.path.join(wandb.run.dir, 'checkpoint.ckpt')
    load_checkpoint_from_cloud(checkpoint_path = 'checkpoint.ckpt')
else:
    checkpoint_path = ckpt_file_location
    
vaegan = VAEGAN.load_from_checkpoint(checkpoint_path)
vaegan = vaegan.eval().to(device)

#latent space walk (currently only for VAE/VAEGAN)

In [ ]:
#obtain latent vector
vae = vaegan.vae

file_loc = [file1,file2]
latent_array = []
for f_loc in file_loc:
    f = Path(f_loc)
    m = trimesh.load(f, force="mesh")
    # m.show()
    m = mesh2arrayCentered(m, array_length=32)
    m = m[np.newaxis,np.newaxis,:,:,:]
    m = torch.Tensor(m).float().type_as(vae.vae_encoder.dis_fc1[0].weight)
    m_rec, z, _ = vae(m, output_all=True)
    latent_array.append(z[0].detach().cpu().numpy())
    m_rec = m_rec.detach().cpu().numpy()
    m_rec = m_rec[0,0,:,:,:]
    m_rec_bool_array = m_rec > 0
    voxelmesh = netarray2mesh(m_rec_bool_array)
    # voxelmesh.show()

In [ ]:
#latent space walk
latent_vectors = np.linspace(latent_array[0], latent_array[1], num=test_num_samples)
# print(latent_vectors.shape)

#genereate meshes from the latent_vectors
latent_vectors = torch.Tensor(latent_vectors).float().type_as(vae.vae_encoder.dis_fc1[0].weight)
meshes = vae.vae_decoder(latent_vectors)[:, 0, :, :, :]
meshes = meshes.detach().cpu().numpy()

#create directory
export_location = Path(export_location)
print(export_location.exists())
if not export_location.exists():
    export_location.mkdir(parents=True)

for n in range(test_num_samples):
    sample_tree_array = meshes[n] > 0
    voxelmesh = netarray2mesh(sample_tree_array)
    export_path = export_location / f"sample_{n}.obj"
    voxelmesh.export(file_obj=export_path, file_type="obj")
print("ok!")